In [128]:
import pandas as pd
import json
from datetime import datetime
import holidays
import os
from datetime import timedelta
import numpy as np

In [129]:
#df filtered by houston, needed to filter by state = TX (NOT IMPLEMENTED)

subway = pd.read_csv('/Users/mazcu/Downloads/Subway_Houston_days.csv', encoding="utf-8")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7,8,9,17,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [130]:
def is_weekend(df):
    df['date'] = pd.to_datetime(df['date']) 

    df['is_weekend'] = df['date'].dt.day_name()
    df.loc[df['is_weekend'] == "Saturday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Sunday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Monday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Tuesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Wednesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Thursday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Friday", 'is_weekend'] = 0

    return df

def is_holiday(df):
    holi = holidays.CountryHoliday('US', prov="Houston", state='TX')
    df["is_holiday"] = [1 if d in holi else 0 for d in df["date"]]
    return df

def last_day_n_week(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        l_week = []
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 1)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')

        week = new['date'] - timedelta(days = 7)
        week = week.dt.strftime('%Y-%m-%d')

        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day, other in zip(yesterday, week):
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
            
            if other in set_raro:
                vis = list(new[new['date'] == other]['visits'])
                l_week.append(vis[0])
            else:
                l_week.append(0)
        
        
        new['visits_last_day'] = l
        new['visits_last_week'] = l_week

        if i == 0:
            old = new
            i += 1
        else:
            old = pd.concat([old, new])
    return old

def income(df):

    cosa = pd.read_csv('data/income.csv', encoding="utf-8", dtype = {"census_block_group": "category"})

    df['poi_cbg'] = df['poi_cbg'].map(int)
    df['poi_cbg'] = df['poi_cbg'].map(str)

    new = df[df['region'] == 'TX']
    cbgs = set(new['poi_cbg'])

    cosa = cosa[cosa['census_block_group'].isin(cbgs)]
    #cosa = cosa[['census_block_group', 'B19013e1']]

    new['cbg_income'] = [0]*91824
    for row in cosa.iterrows():
        new['cbg_income'] = np.where(new['poi_cbg'] == str(row[1][0]), row[1][1], new['cbg_income'])
    
    return new

def rain(df):
    rain_2020 = pd.read_csv('data/rain_houston_2020.csv', sep=';')
    rain_2021 = pd.read_csv('data/rain_houston_2021.csv', sep=';')
    #df['date'] = df['date'].map(int)
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['rain'] = [0]*91824
    l = [rain_2020, rain_2021]
    for rain_df in l:
        for row in rain_df.iterrows():
            df['rain'] = np.where(df['date'] == str(row[1][0]), row[1][1], df['rain'])
        
    return df

def population(df):
    dat = pd.read_csv('data/population.csv')
    cbgs = set(df['poi_cbg'])
    dat = dat[dat['census_block_group'].map(str).isin(cbgs)]
    dat = dat[['census_block_group', 'B00001e1']]
    df['cbg_population'] = [0]*91824
    for row in dat.iterrows():
        df['cbg_population'] = np.where(df['poi_cbg'].map(str) == str(int(row[1][0])), row[1][1], df['cbg_population'])
    return df

subway = is_weekend(subway)
subway = is_holiday(subway)
subway = last_day_n_week(subway)
subway = income(subway)
subway = rain(subway)
subway = population(subway)


<ipython-input-130-e427cda93aea>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_day'] = l
<ipython-input-130-e427cda93aea>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_week'] = l_week
<ipython-input-130-e427cda93aea>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [122]:
subway

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,category_tags,...,year,visits,day,is_weekend,is_holiday,visits_last_day,visits_last_week,cbg_income,rain,cbg_population
806,zzy-222@8fc-ftt-wc5,sg:708e1ba25d1849a89d62078a44221f81,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,2020,2,1,0,1,0,0,19271.0,"0,0",61.0
807,zzy-222@8fc-ftt-wc5,sg:708e1ba25d1849a89d62078a44221f81,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,2020,6,2,0,0,2,0,19271.0,"7,112",61.0
808,zzy-222@8fc-ftt-wc5,sg:708e1ba25d1849a89d62078a44221f81,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,2020,6,3,0,0,6,0,19271.0,"0,7619999999999999",61.0
809,zzy-222@8fc-ftt-wc5,sg:708e1ba25d1849a89d62078a44221f81,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,2020,2,4,1,0,6,0,19271.0,"0,0",61.0
810,zzy-222@8fc-ftt-wc5,sg:708e1ba25d1849a89d62078a44221f81,NaN,NaN,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",...,2020,2,5,1,0,2,0,19271.0,"0,0",61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89753,zzy-223@8fc-ffc-y7q,sg:72d4db38395c4ef292e5a62188778b32,22z-222@8fc-ffc-y7q,sg:3fc3bfd5bb164e49adff737c39dd8b44,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,2021,4,27,1,0,5,7,119000.0,"0,0",324.0
89754,zzy-223@8fc-ffc-y7q,sg:72d4db38395c4ef292e5a62188778b32,22z-222@8fc-ffc-y7q,sg:3fc3bfd5bb164e49adff737c39dd8b44,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,2021,3,28,1,0,4,1,119000.0,"0,0",324.0
89755,zzy-223@8fc-ffc-y7q,sg:72d4db38395c4ef292e5a62188778b32,22z-222@8fc-ffc-y7q,sg:3fc3bfd5bb164e49adff737c39dd8b44,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,2021,5,29,0,0,3,4,119000.0,"1,778",324.0
89756,zzy-223@8fc-ffc-y7q,sg:72d4db38395c4ef292e5a62188778b32,22z-222@8fc-ffc-y7q,sg:3fc3bfd5bb164e49adff737c39dd8b44,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,NaN,NaN,NaN,...,2021,4,30,0,0,5,4,119000.0,"0,0",324.0


import os

for filename in os.listdir('/Users/mazcu/Downloads/safegraph_open_census_data/data/'): 
    print(filename)
    cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/'+filename, encoding="utf-8")
    try:
        print(filename, cosa['B19013e1'])
    except:
        pass

In [84]:
cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b19.csv', encoding="utf-8", dtype = {"census_block_group": "category"})


In [8]:
df = subway

df['poi_cbg'] = df['poi_cbg'].map(int)
df['poi_cbg'] = df['poi_cbg'].map(str)


In [9]:
new = df[df['region'] == 'TX']
cbgs = set(new['poi_cbg'])

In [10]:
cosa = cosa[cosa['census_block_group'].isin(cbgs)]
cosa = cosa[['census_block_group', 'B19013e1']]

In [13]:
def income(df):

    cosa = pd.read_csv('income.csv', encoding="utf-8", dtype = {"census_block_group": "category"})

    df['poi_cbg'] = df['poi_cbg'].map(int)
    df['poi_cbg'] = df['poi_cbg'].map(str)

    new = df[df['region'] == 'TX']
    cbgs = set(new['poi_cbg'])

    cosa = cosa[cosa['census_block_group'].isin(cbgs)]
    #cosa = cosa[['census_block_group', 'B19013e1']]

    df['cbg_income'] = [0]*92796
    for row in cosa.iterrows():
        df['cbg_income'] = np.where(df['poi_cbg'] == str(row[1][0]), row[1][1], df['cbg_income'])

subway = income(subway)

In [18]:
subway['cbg_income']

806       19271.0
807       19271.0
808       19271.0
809       19271.0
810       19271.0
           ...   
89753    119000.0
89754    119000.0
89755    119000.0
89756    119000.0
89757    119000.0
Name: cbg_income, Length: 92796, dtype: float64

In [44]:
rain_2020 = pd.read_csv('rain_houston_2020.csv', sep=';')
rain_2021 = pd.read_csv('rain_houston_2021.csv', sep=';')

In [80]:
def rain(df):
    rain_2020 = pd.read_csv('rain_houston_2020.csv', sep=';')
    rain_2021 = pd.read_csv('rain_houston_2021.csv', sep=';')
    #df['date'] = df['date'].map(int)
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['rain'] = [0]*91824
    l = [rain_2020, rain_2021]
    for rain_df in l:
        for row in rain_df.iterrows():
            df['rain'] = np.where(df['date'] == str(row[1][0]), row[1][1], df['rain'])
        
    return df

In [78]:
subway['date'] = pd.to_datetime(subway['date'])

In [79]:
subway['date'].dt.strftime('%Y-%m-%d')

806      2020-01-01
807      2020-01-02
808      2020-01-03
809      2020-01-04
810      2020-01-05
            ...    
89753    2021-03-27
89754    2021-03-28
89755    2021-03-29
89756    2021-03-30
89757    2021-03-31
Name: date, Length: 91824, dtype: object

In [87]:
for csv in os.listdir('/Users/mazcu/Downloads/safegraph_open_census_data/data/'):
    test = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b00.csv' + csv)
    try:
        print(csv, test['B00001e1'])
    except:
        print(csv, 'NO')

cbg_c24.csv NO
cbg_b20.csv NO
cbg_b08.csv NO
cbg_b09.csv NO
cbg_b21.csv NO
cbg_b23.csv NO
cbg_b22.csv NO
cbg_b27.csv NO
cbg_b19.csv NO
cbg_b25.csv NO
cbg_b01.csv NO
cbg_b15.csv NO
cbg_b14.csv NO
cbg_b00.csv 0          72.0
1         173.0
2         100.0
3         167.0
4         184.0
          ...  
220328     42.0
220329    150.0
220330     64.0
220331    118.0
220332     41.0
Name: B00001e1, Length: 220333, dtype: float64
cbg_b02.csv NO
cbg_b03.csv NO
cbg_b07.csv NO
cbg_b12.csv NO
cbg_patterns.csv NO
cbg_b11.csv NO
cbg_c16.csv NO
cbg_c17.csv NO
cbg_b99.csv NO


In [95]:
def population(df):
    dat = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b00.csv')
    cbgs = set(df['poi_cbg'])
    dat = dat[dat['census_block_group'].map(str).isin(cbgs)]
    dat = dat[['census_block_group', 'B00001e1']]
    df['cbg_population'] = [0]*91824
    for row in dat.iterrows():
        df['cbg_population'] = np.where(df['poi_cbg'].map(str) == str(int(row[1][0])), row[1][1], df['cbg_population'])
    return df

In [103]:
dat = dat[dat['census_block_group'].map(str).isin(cbgs)]
dat = dat[['census_block_group', 'B00001e1']]

In [115]:
subway['cbg_population'] = [0]*91824
for row in dat.iterrows():
    subway['cbg_population'] = np.where(subway['poi_cbg'].map(str) == str(int(row[1][0])), row[1][1], subway['cbg_population'])

In [118]:
dat.to_csv('population.csv', index=False)

In [123]:
nf = pd.read_csv('/Users/mazcu/Downloads/home_panel_summary (3).csv')

nf

,year,month,state,census_block_group,number_devices_residing
0,2021,3,ak,21300001001,24
1,2021,3,ak,21700009001,69
2,2021,3,ak,20200006007,52
3,2021,3,ak,20200001012,78
4,2021,3,ak,20200021004,15
...,...,...,...,...,...
219888,2021,3,wy,560250009011,97
219889,2021,3,wy,560379716001,40
219890,2021,3,wy,560379711001,55
219891,2021,3,wy,560139402024,44


In [124]:
nf = nf[nf['census_block_group'].map(str).isin(cbgs)]

In [125]:
nf

,year,month,state,census_block_group,number_devices_residing
185335,2021,3,tx,482012222001,126
185454,2021,3,tx,482015547001,250
185483,2021,3,tx,482014335012,87
185509,2021,3,tx,482014537002,110
185519,2021,3,tx,482013302001,116
...,...,...,...,...,...
200617,2021,3,tx,482015204001,69
200914,2021,3,tx,482013207002,109
200960,2021,3,tx,482014233012,102
200965,2021,3,tx,482013402012,89


In [131]:
nf = nf[['census_block_group', 'number_devices_residing']]
subway['cbg_devices'] = [0]*91824
for row in nf.iterrows():
    subway['cbg_devices'] = np.where(subway['poi_cbg'].map(str) == str(int(row[1][0])), row[1][1], subway['cbg_devices'])

In [135]:
subway['real_visits'] = (subway['cbg_population']//subway['cbg_devices'])*subway['visits']

In [136]:
subway['real_visits']

806       2.0
807       6.0
808       6.0
809       2.0
810       2.0
         ... 
89753     4.0
89754     3.0
89755     5.0
89756     4.0
89757    10.0
Name: real_visits, Length: 91824, dtype: float64

In [137]:
subway['visits']

806       2
807       6
808       6
809       2
810       2
         ..
89753     4
89754     3
89755     5
89756     4
89757    10
Name: visits, Length: 91824, dtype: int64

In [140]:
subway.iloc[0]

placekey                                                   zzy-222@8fc-ftt-wc5
safegraph_place_id                         sg:708e1ba25d1849a89d62078a44221f81
parent_placekey                                                            NaN
parent_safegraph_place_id                                                  NaN
location_name                                                           Subway
safegraph_brand_ids                  SG_BRAND_de80593878cb1673c62a7f338dc7e4e1
brands                                                                  Subway
top_category                               Restaurants and Other Eating Places
sub_category                                       Limited-Service Restaurants
category_tags                Counter Service,Sandwich Shop,Lunch,Breakfast,...
naics_code                                                            722513.0
latitude                                                             29.611938
longitude                                           